<a href="https://colab.research.google.com/github/8itoonmaria/IAT360_FINAL_PROJECT/blob/main/IAT360_FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>